# 2025 EAI Lab 5

## Topic 1 : From PyTorch To ONNX

### Steps:
1.   Define Model Architecture
2.   Load Weight
3.   Export ONNX File
4.   Quantize To INT8
5.   Building Session



In [1]:
!pip install -U \
    torch torchvision torchaudio \
    onnx onnxscript onnxruntime onnxruntime-tools \
    gradio

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import onnx

# TODO
# Design Your ResNet18 Model

class BasicBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(BasicBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )


    def forward(self, x):
        out = self.left(x)
        out = out + self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet18(nn.Module):
    def __init__(self, ResBlock, num_classes=10):
        super(ResNet18, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = self.make_layer(ResBlock, 64, 2, stride=1)
        self.layer2 = self.make_layer(ResBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)  # First block may have stride > 1, others stride=1
        layers = []
        for s in strides:
            layers.append(block(self.inchannel, channels, s))
            self.inchannel = channels  # Update inchannel for the next block
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [3]:
torch_model = ResNet18(ResBlock=BasicBlock, num_classes=10)
dummy_input = (torch.randn(1, 3, 32, 32),)

def export_onnx(model, dummy, path):
    state = torch.load(path, map_location=torch.device("cpu"))

    # TODO : load state dict
    model.load_state_dict(state, strict=False)
    model.eval()

    # TODO : Export ONNX FILE
    torch.onnx.export(
        model,                     # the model instance
        dummy,                     # example input tuple
        "n26140692_FP32.onnx",     # output ONNX file
        input_names=["input"],     # optional: name of input node
        output_names=["output"],   # optional: name of output node
        opset_version=18           # recommended ONNX opset version
    )
    pass

if __name__ == "__main__":
    export_onnx(model=torch_model, dummy=dummy_input, path="best_model.pth")


[torch.onnx] Obtain model graph for `ResNet18([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ResNet18([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 40 of general pattern rewrite rules.


In [ ]:
# set path
onnx_path = "n26140692_FP32.onnx"
data_path = "n26140692_FP32.onnx.data"
print(f"start repairing: {onnx_path}...")

try:
    # 1. load model
    model = onnx.load(onnx_path)
    print("model loaded...")

    # 2. force overwrite
    onnx.save(model, onnx_path)
    print("model merged...")

    # 3. delete reduntant *.data file
    if os.path.exists(data_path):
        os.remove(data_path)
        print(f"reduntant file deleted...: {data_path}")
    else:
        print("no *.data found")

    # 4. final confirmation
    size_mb = os.path.getsize(onnx_path) / (1024 * 1024)
    print(f"file size after repairing {size_mb:.2f} MB")
    
    if size_mb > 40:
        print("done repairing !")
    else:
        print("warning! file still unnormally small")

except Exception as e:
    print(f"failed repairing: {e}")

🔧 開始修復模型: n26140692_FP32.onnx
✅ 模型載入成功 (包含外部資料)
✅ 模型已重新儲存為單一檔案
🗑️ 已刪除多餘的資料檔: n26140692_FP32.onnx.data
📊 修復後檔案大小: 42.65 MB
🎉 修復完成！現在 Gradio 應該可以讀取了。


In [5]:
import os, numpy as np
from PIL import Image
import onnxruntime as ort
from onnxruntime.quantization import CalibrationDataReader

CIFAR10_MEAN = np.array([0.4914, 0.4822, 0.4465], dtype=np.float32)
CIFAR10_STD  = np.array([0.2470, 0.2435, 0.2616], dtype=np.float32)

def preprocess_32x32(pil_img: Image.Image) -> np.ndarray:
    arr = np.asarray(pil_img.convert("RGB").resize((32, 32)), dtype=np.float32) / 255.0
    arr = (arr - CIFAR10_MEAN) / CIFAR10_STD
    return arr.transpose(2, 0, 1)[None, ...]  # (1,3,32,32)

class CIFARLikeCalibReader(CalibrationDataReader):
    def __init__(self, image_dir: str = None, input_name: str = "input",
                 batch_size: int = 32, num_batches: int = 10):
        self.input_name  = input_name
        self.batch_size  = batch_size
        self.num_batches = num_batches
        self.paths = []
        if image_dir and os.path.isdir(image_dir):
            for f in os.listdir(image_dir):
                if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
                    self.paths.append(os.path.join(image_dir, f))
        self._mode_random = len(self.paths) == 0
        self._pos = 0
        self._emitted = 0

    def get_next(self):
        if self._emitted >= self.num_batches:
            return None
        if self._mode_random:
            batch = np.random.randn(self.batch_size, 3, 32, 32).astype(np.float32)
        else:
            items = []
            for _ in range(self.batch_size):
                if self._pos >= len(self.paths):
                    break
                img = Image.open(self.paths[self._pos])
                self._pos += 1
                items.append(preprocess_32x32(img))
            if not items:
                return None
            batch = np.concatenate(items, axis=0).astype(np.float32)
        self._emitted += 1
        return {self.input_name: batch}

    def rewind(self):
        self._pos = 0
        self._emitted = 0

FP32_MODEL = "n26140692_FP32.onnx"
INT8_MODEL = "n26140692_INT8.onnx"


_tmp = ort.InferenceSession(FP32_MODEL, providers=["CPUExecutionProvider"])
INPUT_NAME = _tmp.get_inputs()[0].name
print("Calib will use input name:", INPUT_NAME)


Calib will use input name: input


In [6]:
from onnxruntime.quantization import quantize_static, QuantType, CalibrationMethod
from onnxruntime.quantization.preprocess import quant_pre_process

reader = CIFARLikeCalibReader(
    image_dir=None,
    input_name=INPUT_NAME,
    batch_size=1,
    num_batches=50
)

def quantize_to_int8(fp32_path, int8_path, reader, method="MinMax"):
    preprocessed_model_path = fp32_path.replace(".onnx", "_pre.onnx")

    print("Step 1: Running Model Pre-processing...")

    quant_pre_process(
        input_model_path=fp32_path,
        output_model_path=preprocessed_model_path
    )

    print("Step 2: Running Quantization...")
    # Todo : quantize_static
    quantize_static(
        model_input=preprocessed_model_path,
        model_output=int8_path,
        calibration_data_reader=reader,
        quant_format=ort.quantization.QuantFormat.QDQ,
        per_channel=True,
        weight_type=QuantType.QInt8,
        activation_type=QuantType.QUInt8,
        reduce_range=False,
        calibrate_method=CalibrationMethod.MinMax,
        op_types_to_quantize=["Conv","MatMul","Gemm"]
    )
    print("Saved INT8 model:", INT8_MODEL)

quantize_to_int8(FP32_MODEL, INT8_MODEL, reader)

Step 1: Running Model Pre-processing...
Step 2: Running Quantization...
Saved INT8 model: n26140692_INT8.onnx


In [7]:
import time
import numpy as np
import onnxruntime as ort

def run(sess, x):
    return sess.run(None, {sess.get_inputs()[0].name: x})[0]

x_demo = np.random.randn(1,3,32,32).astype(np.float32)

# Todo : build session function
def build_session(model_path, providers):
  sess = ort.InferenceSession(model_path, providers=providers)
  return sess



sess_fp32 = build_session(model_path=FP32_MODEL, providers=["CPUExecutionProvider"])
sess_int8 = build_session(model_path=INT8_MODEL, providers=["CPUExecutionProvider"])

y_fp32 = run(sess_fp32, x_demo)
y_int8 = run(sess_int8, x_demo)

l2_rel = np.linalg.norm(y_fp32 - y_int8) / (np.linalg.norm(y_fp32) + 1e-12)
print(f"[Check] relative L2 diff FP32 vs INT8: {l2_rel:.6f}")

def bench(sess, x, n=50):
    t0 = time.time()
    for _ in range(n):
        sess.run(None, {sess.get_inputs()[0].name: x})
    return (time.time() - t0) / n

print("FP32 avg sec:", bench(sess_fp32, x_demo))
print("INT8 avg sec:", bench(sess_int8, x_demo))

so = ort.SessionOptions()
so.enable_profiling = True



[Check] relative L2 diff FP32 vs INT8: 0.013204
FP32 avg sec: 0.006542115211486816
INT8 avg sec: 0.0020003461837768553


## Topic 2 : Gradio


In [8]:
! pip install gradio

In [9]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import gradio as gr
import time

# ====== Config ======
MODEL_PATH_INT8 = "n26140692_INT8.onnx"   # INT8 ONNX Model
MODEL_PATH_FP32 = "n26140692_FP32.onnx"     # FP32 ONNX Model
LABELS = ['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']

# CIFAR-10 Normalization Parameter
CIFAR10_MEAN = np.array([0.4914, 0.4822, 0.4465], dtype=np.float32)
CIFAR10_STD  = np.array([0.2470, 0.2435, 0.2616], dtype=np.float32)

# ====== Utils ======
def softmax_np(x: np.ndarray) -> np.ndarray:
    x = x - np.max(x)
    ex = np.exp(x)
    return ex / np.sum(ex)

# TODO : preprocess input image function
def preprocess(image: Image.Image) -> np.ndarray:
    """Input PIL Image → (1,3,32,32) float32"""
    if not isinstance(image, Image.Image):
        raise ValueError("Please Upload Image")

    # 1. Resize to 32x32 & Convert to RGB
    img = image.convert("RGB").resize((32, 32))
    
    # 2. To Numpy & Scale to [0, 1]
    arr = np.array(img, dtype=np.float32) / 255.0
    
    # 3. Normalize (Standardization)
    arr = (arr - CIFAR10_MEAN) / CIFAR10_STD
    
    # 4. Transpose (H, W, C) -> (C, H, W)
    arr = arr.transpose(2, 0, 1)
    
    # 5. Add Batch Dimension -> (1, C, H, W)
    arr = np.expand_dims(arr, 0)

    return arr

# ====== ONNX Sessions ======
# providers = ort.get_available_providers()
providers = ['CPUExecutionProvider']

sess_int8 = build_session(MODEL_PATH_INT8, providers=providers)
in_int8  = sess_int8.get_inputs()[0].name
out_int8 = sess_int8.get_outputs()[0].name


try:
    sess_fp32 = build_session(MODEL_PATH_FP32, providers=providers)
    in_fp32  = sess_fp32.get_inputs()[0].name
    out_fp32 = sess_fp32.get_outputs()[0].name
    _fp32_err = ""
except Exception as e:
    sess_fp32, in_fp32, out_fp32 = None, None, None
    _fp32_err = f"[FP32 load failure] {type(e).__name__}: {e}"

# ====== Compare FP32 and INT8 ======
# TODO : Compare FP32 and INT8
def compare_fp32_int8(image: Image.Image):
    if image is None:
        return {}, {}, "Please Upload Your Image。"
    if sess_fp32 is None:
        return {}, {}, (_fp32_err or "The FP32 model has not been provided, so a comparison cannot be made.")

    x = preprocess(image)

    # Your progarm
    # --- Run FP32 ---
    t0 = time.time()
    res_fp32 = sess_fp32.run([out_fp32], {in_fp32: x})[0].squeeze().astype(np.float32)
    t1 = time.time()
    fp32_ms = (t1 - t0) * 1000

    # --- Run INT8 ---
    t0 = time.time()
    res_int8 = sess_int8.run([out_int8], {in_int8: x})[0].squeeze().astype(np.float32)
    t1 = time.time()
    int8_ms = (t1 - t0) * 1000


    p_fp32 = softmax_np(res_fp32)
    p_int8 = softmax_np(res_int8)

    def top3_map(p):
        idx = np.argpartition(p, -3)[-3:]
        idx = idx[np.argsort(p[idx])[::-1]]
        return {LABELS[i]: float(p[i]) for i in idx}

    top3_fp32 = top3_map(p_fp32)
    top3_int8 = top3_map(p_int8)

    speedup = fp32_ms / int8_ms if int8_ms > 0 else 0
    summary = (
        f"FP32 inference time: {fp32_ms:.2f} ms\n"
        f"INT8 inference time: {int8_ms:.2f} ms\n"
        f"Speedup (FP32/INT8): {speedup:.2f}×"
    )
    return top3_fp32, top3_int8, summary

# ====== Gradio UI ======
# TODO : Building GUI Interface
demo = gr.Interface(
    fn = compare_fp32_int8,
    inputs = gr.Image(type="pil", label="Upload Image"),
    outputs = [
        gr.Label(num_top_classes=3, label="FP32 Prediction (Original)"),
        gr.Label(num_top_classes=3, label="INT8 Prediction (Quantized)"),
        gr.Textbox(label="Performance Comparison")
    ],
    title = "ONNX Quantization Demo: FP32 vs INT8",
    description = "Compare the accuracy and speed of the original ResNet model vs. the Quantized INT8 model on CIFAR-10."
)

if __name__ == "__main__":
    # TODO : building a public web
    demo.launch(share=True)


/Users/jackiempty/Git_repo/AISLab/EAI_Lab/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://b3249f1bef931d3037.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
